<a href="https://colab.research.google.com/github/jny0812/trading_project/blob/main/%08trading_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1) 모듈 import

In [ ]:
!pip3 install -U finance-datareader
!pip3 install pykrx

In [ ]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from pykrx import stock
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

## 크롤링

In [ ]:
stockList = fdr.StockListing('KRX')

def crawling(page, kospi100):
    res = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(page))
    soup = BeautifulSoup(res.content,'html.parser')
    
    #section = soup.select('tbody')
    #items= section.find_all('tr', onmouseover="mouseOver(this)")
    
    items = soup.select('tbody > tr > td:nth-of-type(2) > a')
    
    for item in range(len(items)):
        name=items[item].contents[0]
        if (stockList[stockList['Name']==name]['ListingDate']<'2018-1-1').tolist():
            code=items[item].attrs['href'][-6:]
            kospi100[code]=name
            
            if len(kospi100)==100:
                break
                
    return kospi100

def Kospi_100(): #kospi 상위 100개 출력 함수
    
    page = 1
    kospi100=dict()
    
    while(len(kospi100)!=100):  
        kospi100=crawling(page,kospi100)
        page=page+1
        
    return kospi100


In [ ]:
kospi100 = Kospi_100()

kospi100

In [ ]:
kospi_df = pd.DataFrame()

kospi_df['Code'] = kospi100.keys()
kospi_df['Name'] =kospi100.values()

kospi_df

In [ ]:
def trading(code,start_date,end_date):
    df = fdr.DataReader(code,start_date, end_date)

    return df

In [ ]:
#1) before corona

for i in range(len(kospi_df)):
    code = kospi_df['Code'][i]
    df_kospi = trading(code,'20180101','20191231')
    
df_kospi

###  전략: 주가가 20일 이평선을 넘어가면 매수 & 주가 하락하는 시점에 바로 매도 (100주) & initial = 1억
